In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import os
import tensorflow as tf
import sys
import seaborn as sns
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import minmax_scale
from tensorflow import keras
import random

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_row', 100)

from keras import backend as K
K.clear_session()
tf.compat.v1.reset_default_graph()

In [2]:
Total_Data_add_Gender = np.load('./Data/Full_Data_After_Pre.npy')
Cal_Total_Data_add_Gender = np.load('./Data/Cal_Full_Data_After_Pre.npy')
Total_Data_Gender = np.load('./Data/Total_Data_Gender.npy')
Cal_Total_Data_Gender = np.load('./Data/Cal_Total_Data_Gender.npy')
Total_Data_std = np.load('./Data/Total_Data_std.npy')
Total_Data_mean =np.load('./Data/Total_Data_mean.npy')

print('Full_Data_shape : ', Total_Data_add_Gender.shape, 'Cal_Full_Data_Shape : ', Cal_Total_Data_add_Gender.shape)
print('Gender_shape : ', Total_Data_Gender.shape, 'Cal_Gender_shape : ', Cal_Total_Data_Gender.shape)


# test

Total_Data_add_Gender_Test = np.load('./Data/Full_Data_After_Pre_Test.npy')
Cal_Total_Data_add_Gender_Test = np.load('./Data/Cal_Full_Data_After_Pre_Test.npy')
Total_Data_Gender_Test = np.load('./Data/Total_Data_Gender_Test.npy')
Cal_Total_Data_Gender_Test = np.load('./Data/Cal_Total_Data_Gender_Test.npy')
Total_Data_std_Test = np.load('./Data/Total_Data_std_Test.npy')
Total_Data_mean_Test = np.load('./Data/Total_Data_mean_Test.npy')

print('Full_Data_shape : ', Total_Data_add_Gender_Test.shape, 'Cal_Full_Data_Shape : ', Cal_Total_Data_add_Gender_Test.shape)
print('Gender_shape : ', Total_Data_Gender_Test.shape, 'Cal_Gender_shape : ', Cal_Total_Data_Gender_Test.shape)

Full_Data_shape :  (1042, 11) Cal_Full_Data_Shape :  (1042, 11)
Gender_shape :  (1042,) Cal_Gender_shape :  (1042,)
Full_Data_shape :  (1042, 11) Cal_Full_Data_Shape :  (1042, 11)
Gender_shape :  (1042,) Cal_Gender_shape :  (1042,)


# test

# d

In [3]:
def make_noise():
    random0 = np.random.randn(len(Total_Data_add_Gender)).reshape(-1, 1)

    random1 = np.random.power(6, len(Total_Data_add_Gender)).reshape(-1, 1)

    random2 = np.random.randn(len(Total_Data_add_Gender)).reshape(-1, 1)

    random3 = np.random.randn(len(Total_Data_add_Gender)).reshape(-1, 1)

    random4 = np.random.randn(len(Total_Data_add_Gender)).reshape(-1, 1)

    random5 = np.random.exponential(scale = 1.0, size = len(Total_Data_add_Gender)).reshape(-1, 1)
    random5 = minmax_scale(random5, feature_range=(0, 1), axis = 0, copy = True)

    random6 = np.random.exponential(scale = 1.0, size = len(Total_Data_add_Gender)).reshape(-1, 1)
    random6 = minmax_scale(random6, feature_range=(0, 15), axis = 0, copy = True)

    random7 = np.random.randn(len(Total_Data_add_Gender)).reshape(-1, 1)

    random8 = np.random.randn(len(Total_Data_add_Gender)).reshape(-1, 1)

    temp1 = []
    random9 = np.eye(2)[np.random.choice(2, len(Total_Data_add_Gender))]
    for i in random9:
        temp1.append(i[0])
    temp1 = np.array(temp1).reshape(-1, 1)
    random9 = temp1

    temp2 = []
    random10 = np.eye(2)[np.random.choice(2, len(Total_Data_add_Gender))]
    for i in random10:
        temp2.append(i[0])
    temp2 = np.array(temp2).reshape(-1, 1)
    random10 = temp2
    

    concat_noise = np.concatenate((random0, random1, random2, random3, random4, random5, random6, random7, random8, random9, random10), axis = 1)
    
    return concat_noise

In [4]:
generator = keras.models.Sequential([
    keras.layers.Dense(30, input_shape=[11]),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha = 0.2),
        keras.layers.Dense(60),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha = 0.2),
        keras.layers.Dense(30),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha = 0.2),
        keras.layers.Dense(11)
])

discriminator = keras.models.Sequential([
    keras.layers.Dense(30),
        keras.layers.LeakyReLU(alpha = 0.2),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(60),
        keras.layers.LeakyReLU(alpha = 0.2),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(30),
        keras.layers.LeakyReLU(alpha = 0.2),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(11)
])
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="adam")

In [5]:
def train_gan(gan, dataset, n_epochs=500):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
    # phase 1 - training the discriminator
        noise_ = make_noise()
        generated_data = generator(noise_)
        X_fake_and_real = tf.concat([generated_data, dataset], axis=0)
        y1 = tf.constant([[0.]] * len(dataset) + [[1.]] * len(dataset))
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X_fake_and_real, y1)
            
    # phase 2 - training the generator
        _noise = make_noise()
        y2 = tf.constant([[1.]] * len(dataset))
        discriminator.trainable = False
        g_loss = gan.train_on_batch(_noise, y2)
        if epoch % 50 == 0:
            print('epoch : ', epoch, 'd_loss : ', d_loss, 'g_loss ', g_loss)

In [6]:
train_gan(gan, Total_Data_add_Gender_Test)

epoch :  0 d_loss :  3.8703360557556152 g_loss  7.576006889343262
epoch :  50 d_loss :  1.5132267475128174 g_loss  2.30733323097229
epoch :  100 d_loss :  1.8155854940414429 g_loss  1.6666375398635864
epoch :  150 d_loss :  1.648278832435608 g_loss  2.074953317642212
epoch :  200 d_loss :  1.7210766077041626 g_loss  2.3437812328338623
epoch :  250 d_loss :  1.7871865034103394 g_loss  2.452660322189331
epoch :  300 d_loss :  1.623934268951416 g_loss  2.0529754161834717
epoch :  350 d_loss :  1.4680511951446533 g_loss  2.0802109241485596
epoch :  400 d_loss :  1.515739917755127 g_loss  1.7153747081756592
epoch :  450 d_loss :  1.5745553970336914 g_loss  1.478927493095398


In [7]:
Cal_generated_Data_Samples_Test = generator(Cal_Total_Data_add_Gender_Test, training = False)

In [8]:
for i, value in enumerate(Cal_Total_Data_add_Gender_Test):
    gene_Test = []
    gene_Test = value[:-2]
    for j, value2 in enumerate(gene_Test):
        if value2 == 0:
            Cal_Total_Data_add_Gender_Test[i][j] = Cal_generated_Data_Samples_Test[i][j]
            
print('shape : ', Cal_Total_Data_add_Gender_Test.shape)
print()
print('Data : ', Cal_Total_Data_add_Gender_Test[0])

shape :  (1042, 11)

Data :  [ 0.765507    0.252955   -0.63193885 -1.2543553   0.70166014 -1.15055728
  0.11156088  0.22818987 -0.48356657  1.          0.        ]


# Rescaling

In [9]:
Total_Data_add_Gender_Test__Test = np.delete(Total_Data_add_Gender_Test, (9, 10), axis = 1)
Total_Data_add_Gender_Test_v2_Test = []

for i, value in enumerate(Total_Data_add_Gender_Test__Test):
    _temp_ = []
    for j, value in enumerate(value):
        _temp_.append(value)
    _temp_.append(Total_Data_Gender_Test[i])
    Total_Data_add_Gender_Test_v2_Test.append(_temp_)
    
Total_Data_add_Gender_Test_v2_Test = np.array(Total_Data_add_Gender_Test_v2_Test)
Total_Data_add_Gender_Test = Total_Data_add_Gender_Test_v2_Test

In [10]:
rescaling_data1 = []

for i, value1 in enumerate(Total_Data_add_Gender_Test):
    new_data = []
    c_value = value1[:-1]
    v_value = value1[-1]
    for j, value2 in enumerate(c_value):
        _temp_ = value2 * Total_Data_std_Test[j] + Total_Data_mean_Test[j]
        new_data.append(_temp_)
    
    new_data.append(v_value)
    rescaling_data1.append(new_data)
    
rescaling_data1 = np.array(rescaling_data1)
rescaling_data1 = abs(rescaling_data1)
rescaling_data1.shape

(1042, 10)

In [11]:
rescaling_data1[:5]

array([[124.  ,  92.5 ,  38.75,  89.  ,  30.  ,   3.7 ,   8.34,  52.  ,
         66.89,   1.  ],
       [ 97.  , 100.  ,  36.5 , 115.  ,  12.5 ,   0.5 ,   9.6 , 414.  ,
         63.59,   0.  ],
       [ 73.  , 100.  ,  37.9 , 116.  ,  23.  ,   0.9 ,  11.3 , 167.  ,
         49.38,   1.  ],
       [ 84.  ,  86.  ,  38.44, 117.  ,  31.  ,   0.5 ,   9.1 , 211.  ,
         77.56,   1.  ],
       [ 87.  ,  96.  ,  36.78,  99.  ,  24.  ,   0.7 ,  11.4 , 341.  ,
         71.24,   1.  ]])

In [12]:
Cal_Total_Data_add_Gender_Test__Test = np.delete(Cal_Total_Data_add_Gender_Test, (9, 10), axis = 1)
Cal_Total_Data_add_Gender_Test_v2_Test = []

for i, value in enumerate(Cal_Total_Data_add_Gender_Test__Test):
    _temp_ = []
    for j, value in enumerate(value):
        _temp_.append(value)
    _temp_.append(Cal_Total_Data_Gender_Test[i])
    Cal_Total_Data_add_Gender_Test_v2_Test.append(_temp_)
    
Cal_Total_Data_add_Gender_Test_v2_Test = np.array(Cal_Total_Data_add_Gender_Test_v2_Test)
Cal_Total_Data_add_Gender_Test = Cal_Total_Data_add_Gender_Test_v2_Test

In [13]:
Cal_Total_Data_add_Gender_Test.shape

(1042, 10)

In [14]:
rescaling_data2 = []

for i, value1 in enumerate(Cal_Total_Data_add_Gender_Test):
    new_data = []
    c_value = value1[:-1]
    v_value = value1[-1]
    for j, value2 in enumerate(c_value):
        _temp_ = value2 * Total_Data_std_Test[j] + Total_Data_mean_Test[j]
        new_data.append(_temp_)
    
    new_data.append(v_value)
    rescaling_data2.append(new_data)
    
rescaling_data2 = np.array(rescaling_data2)
rescaling_data2 = abs(rescaling_data2)
rescaling_data2 = np.around(rescaling_data2, 2)
rescaling_data2.shape

(1042, 10)

In [15]:
rescaling_data2[:5]

array([[1.0300e+02, 9.8000e+01, 3.6400e+01, 9.2000e+01, 2.3000e+01,
        5.3000e-01, 1.2950e+01, 2.0601e+02, 5.5000e+01, 0.0000e+00],
       [9.0000e+01, 9.1500e+01, 3.7850e+01, 1.1950e+02, 1.6000e+01,
        2.1700e+00, 1.7390e+01, 3.6399e+02, 6.6000e+01, 1.0000e+00],
       [8.9000e+01, 9.8000e+01, 3.7670e+01, 1.0500e+02, 1.8000e+01,
        1.0000e-01, 1.1910e+01, 2.6516e+02, 4.5960e+01, 1.0000e+00],
       [9.3000e+01, 9.7000e+01, 3.6900e+01, 1.5100e+02, 1.4000e+01,
        1.4000e+00, 1.9400e+01, 2.2417e+02, 6.8000e+01, 1.0000e+00],
       [7.3000e+01, 1.0000e+02, 3.7100e+01, 1.3700e+02, 1.7000e+01,
        7.1000e-01, 1.5420e+01, 1.2226e+02, 5.2390e+01, 0.0000e+00]])

In [16]:
np.save('C:/jupyter/Human_Deep/Human_Deep/Data/Total_Data_After_Preprocessing', Total_Data_add_Gender_Test)
np.save('C:/jupyter/Human_Deep/Human_Deep/Data/Cal_Total_Data_After_Preprocessing', Cal_Total_Data_add_Gender_Test)

In [16]:
gan.save('gan_saved_model/')
generator.save('g_saved_model/')
discriminator.save('d_saved_model/')

INFO:tensorflow:Assets written to: gan_saved_model/assets
INFO:tensorflow:Assets written to: g_saved_model/assets
INFO:tensorflow:Assets written to: d_saved_model/assets
